In [1]:
from importlib import import_module
from utils import read_input_batch
from collections import deque, defaultdict
from itertools import product, chain
import re

main = import_module("2024.24.main")

In [2]:
filename = "2024/24/input.txt"
OP_DICT = {
    "OR": lambda x, y: x | y,
    "AND": lambda x, y: x & y,
    "XOR": lambda x, y: x ^ y,
}

wires = {
    k: int(v)
    for k, v in dict(
        line.split(": ") for line in read_input_batch(filename, line_split=False)[0]
    ).items()
}

gates = dict()
for gate in read_input_batch(filename, line_split=False)[1]:
    in_1, op, in_2, out = re.findall(r"[a-zA-Z0-9]{2,3}", gate)
    gates[out] = (op, frozenset((in_1, in_2)))

In [3]:
def zero_pad(x: int) -> str:
    return str(x).zfill(2)


def get_n_bit(x: int, n: int) -> int:
    return (x >> n) & 1


def get_inputs(s: str, gates) -> list[str]:
    return sorted([gate for gate in gates if gate[0] == s])




In [4]:
def construct_half_adder(bit):
    adder = dict()
    digit = zero_pad(bit)
    inputs = frozenset((f"x{digit}", f"y{digit}"))
    adder[("XOR", inputs)] = f"z{digit}"
    adder[("AND", inputs)] = f"c{digit}"
    return adder


def construct_full_adder(bit):
    adder = dict()
    digit = zero_pad(bit)
    prev_digit = zero_pad(bit - 1)
    inputs = frozenset((f"x{digit}", f"y{digit}"))

    adder[("XOR", inputs)] = f"a0{digit}"
    adder[("XOR", frozenset((f"a0{digit}", f"c{prev_digit}")))] = f"z{digit}"
    adder[("AND", frozenset((f"a0{digit}", f"c{prev_digit}")))] = f"a1{digit}"
    adder[("AND", inputs)] = f"a2{digit}"
    adder[("OR", frozenset((f"a1{digit}", f"a2{digit}")))] = f"c{digit}"
    return adder


class AdderConstructor:
    def __init__(self, gates):
        self.raw_gates = dict(gates)
        self.gates = {v: k for k, v in gates.items()}
        self.initialize()

    def initialize(self):
        z_bits = get_inputs("z", self.raw_gates)
        self.max_bit = int(z_bits[-1][1:])
        self.mapping = dict()
        for i in range(self.max_bit-1):
            digit = zero_pad(i)
            for s in ('x', 'y'):
                self.mapping[f'{s}{digit}'] = f'{s}{digit}'


    def remap_gates(self, old_key):
        remap_dict = dict()
        for (op, inputs), key in self.gates.items():
            if any(i == old_key for i in inputs):
                new_inputs = frozenset(self.mapping[i] if i == old_key else i for i in inputs)
                remap_dict[(op, new_inputs)] = key
        self.gates.update(remap_dict)




    def assign_gates(self, bit):
        if bit == 0:
            valid_gates = construct_half_adder(bit)
        else:
            valid_gates = construct_full_adder(bit)
        
        while valid_gates:
            if matches:=set(valid_gates).intersection(set(self.gates)):
                for gate in matches:
                    old_key = self.gates[gate]
                    new_key = valid_gates[gate]
                    self.mapping[old_key] = new_key
                    self.remap_gates(old_key)
                    self.gates[gate] = new_key
                    del valid_gates[gate]
        
            else:
                print(bit, 'PANIC')
                return valid_gates.keys()
        return dict().keys()
        

            
def swap(s, pair):
    in_1, in_2 = pair
    if s == in_1:
        return in_2
    elif s == in_2:
        return in_1
    else:
        return s

In [6]:
import itertools

max_bit = int(get_inputs("z", gates)[-1][1:])
pairs = []
raw_gates = dict()
for gate in read_input_batch(filename, line_split=False)[1]:
    in_1, op, in_2, out = re.findall(r"[a-zA-Z0-9]{2,3}", gate)
    raw_gates[out] = (op, frozenset((in_1, in_2)))



queue = deque([(AdderConstructor(raw_gates), dict(raw_gates), [])])
visited = set()
while queue:
    constructor, gates, pairs = queue.pop()
    for bit in range(max_bit):
        print(bit)
        # Assign gates for bit
        test = constructor.assign_gates(bit)
        # If the process fails, find the missing gates
        if test:
            print(test)
            missing_gates = set(list(itertools.chain.from_iterable(v for _, v in test)))
            candidate_keys = set(
                itertools.chain.from_iterable(
                    inputs.difference(missing_gates)
                    for (_, inputs), _ in constructor.gates.items()
                    if missing_gates.intersection(inputs)
                )
            )
            # Iterate over all pairs
            for pair in itertools.combinations(candidate_keys, 2):
                if pair not in visited:
                    visited.add(pair)
                    new_gates = {swap(k, pair): v for k, v in gates.items()}
                    queue.appendleft((AdderConstructor(new_gates), new_gates, pairs + [pair]))
            break
        else:
            continue
    
    else:
        print(','.join(sorted(list(k for k,v in gates.items() for kk, vv in raw_gates.items() if (v ==vv) & (k != kk)))))
        break

0
1
2
3
4
5
6
7
8
9
9 PANIC
dict_keys([('OR', frozenset({'a209', 'a109'}))])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
20 PANIC
dict_keys([('OR', frozenset({'a120', 'a220'}))])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
31 PANIC
dict_keys([('XOR', frozenset({'a031', 'c30'})), ('AND', frozenset({'a031', 'c30'})), ('OR', frozenset({'a131', 'a231'}))])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
31 PANIC
dict_keys([('XOR', frozenset({'a031', 'c30'})), ('AND', frozenset({'a031', 'c30'})), ('OR', frozenset({'a131', 'a231'}))])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
31 PANIC
dict_keys([('XOR', frozenset({'a031', 'c30'})), ('AND', frozenset({'a031', 'c30'})), ('OR', frozenset({'a131', 'a231'}))])
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
31 PANIC
dict_keys([('XOR', frozenset({'a031', 'c30'})), ('AND', frozens

In [8]:
queue

deque([])